# 要約 
このJupyterノートブックは、「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」に参加するための機械学習モデルを構築することを目的としています。具体的には、大規模言語モデル（LLM）が生成した応答の中から、どちらがユーザーに好まれるかを予測するモデルを開発しています。

### 問題の取り組み
ノートブックでは、トレーニングデータとテストデータを読み込み、ユーザーの選好に基づく勝者モデルの確率を予測します。データセットはCSVフォーマットであり、どちらのモデルが選ばれるか、または引き分けになるかの情報を含んでいます。最終的な目標は、各応答がどれだけ好まれるかを予測する確率を出力することです。

### 使用技術とライブラリ
#### 使用ライブラリ:
- **TensorFlow**: 機械学習のためのフレームワークで、モデルの構築やトレーニングに使用されています。
- **Transformers**: 自然言語処理のためのモデルとトークナイザを扱うライブラリ。特にBERTモデルが使用されています。
- **Datasets**: データセットの読み込みと管理を行うためのライブラリ。
- **Pandas**: データの操作と分析を行うためのツール。
- **Matplotlib**: データの可視化を行うライブラリ。

#### 手法:
1. **データの読み込み**: Kaggleから提供されたトレーニングとテストのデータセットを読み込みます。
2. **データの前処理**: 不足している列の追加やデータ型の変換を行います。
3. **トークナイゼーション**: BERTトークナイザを使用して、モデルの入力に合わせたデータのトークナイゼーションを行います。
4. **TensorFlow Datasetの生成**: tf.data.Datasetに変換し、シャッフルやバッチ化を行います。
5. **モデル構築**: Kerasを用いてカスタムBERTモデルを構築します。BERTの出力を使用し、最終的に出力層で選好モデルのクラスを予測します。
6. **モデルのトレーニング**: 指定したエポック数でモデルをトレーニングし、その進捗を表示します。
7. **予測と結果の保存**: テストデータセットに基づいて予測を行い、その結果をCSVファイルとして保存します。

このプロジェクトは、ユーザーの選好を理解し、それに基づいてより良い応答を生成するための基盤を提供します。最終的には、出力された確率を基に、どのモデルの応答が好まれるかを評価することが可能になります。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する専門用語の簡単な解説です。初心者な方がつまずくかもしれないが、基本的な知識を持っている方には有用な情報に絞って説明します。

### 専門用語解説

1. **TPU (Tensor Processing Unit)**:
   - Googleが開発した、特に機械学習のために最適化されたプロセッサです。TPUは大規模な行列計算を高速に行うことができ、ディープラーニングのトレーニングや推論で使用されることが多いです。

2. **TPUStrategy**:
   - TensorFlowでTPUを使用するための分散処理戦略です。複数のTPUデバイスを利用して、トレーニングを並列化するための設定が含まれます。

3. **DataCollatorWithPadding**:
   - バッチ内の異なる長さのシーケンスをパディング（同じ長さに揃えること）するためのユーティリティです。トークナイゼーションの結果、異なる長さの入力が生成される場合に、ミニバッチで処理するために必要です。

4. **tf.data.Dataset**:
   - TensorFlowでデータを効率的に処理するためのオブジェクトで、データのバッチ化、シャッフル、前処理などを効率良く行うことができるデータ構造です。

5. **attention_mask**:
   - トランスフォーマーアーキテクチャで使用されるマスクで、モデルがトークンの入力に対してどのように注目（attention）するかを指定します。主にパディングされたトークンを無視するために使われます。

6. **Lambda層**:
   - Kerasにおける特殊な層で、ユーザー定義の関数を実行するために使用されます。ここでは、BERTの出力から特定のトークン（最初のトークン）を抽出するために使われています。

7. **スパースカテゴリカルクロスエントロピー**:
   - 複数のクラスを持つ分類問題で使用される損失関数。ラベルが整数形式で表現され、クラス間の確率的な誤差を計算します。

8. **カスタムモデル**:
   - ユーザーのニーズに応じて独自に設計したモデルのこと。Kerasのサブクラス化を使用して、特定の前処理や出力の要件に合わせたモデルを構築することができます。

9. **tokenize_function**:
   - 入力テキストをトークナイズするための関数で、テキストをトークンに変換し、必要なパディングや切り捨て処理を行います。ここでは、BERT用のトークナイザを使用して二つのモデルの入出力を処理します。

10. **モデルカード**:
    - モデルのメタデータや使用方法、特長、注意点などを説明するために用意されたドキュメント。実行時の詳細や推論の結果を理解するために役立ちます。

これらの用語は、実際に機械学習やディープラーニングを行う上で重要な概念であり、特にこのノートブックの特定の文脈では理解を深めるのに役立つと思われます。

---


In [ ]:
# osモジュールをインポート
# オペレーティングシステムに関する機能を利用します
import os

# tensorflowライブラリをインポート
# TensorFlowは機械学習のためのフレームワークです
import tensorflow as tf

# datasetsモジュールからload_datasetとDatasetDictをインポート
# データセットを簡単にロードしたり、管理したりするためのものです
from datasets import load_dataset, DatasetDict

# transformersライブラリから必要なクラスをインポート
# 自然言語処理(NLP)のためのモデルやトークナイザを使うためです
from transformers import BertTokenizer, TFBertModel, DataCollatorWithPadding

# Kerasのレイヤーをインポート
# 様々なレイヤーを神経ネットワークのモデルに追加するために使用します
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D, Lambda, Layer, Input, Dropout

# Kerasのモデルをインポート
# モデル構築やトレーニングのために使用します
from tensorflow.keras.models import Model

# shutilモジュールをインポート
# ファイルの操作を簡単にするために使用されます
import shutil

# pandasライブラリをインポート
# データ操作や分析のための強力なツールです
import pandas as pd

# tqdm.kerasからTqdmCallbackをインポート
# 学習プロセスの進捗を表示するために使用されるコールバックです
from tqdm.keras import TqdmCallback

# matplotlibのpyplotをインポート
# データの可視化を行うためのライブラリです
import matplotlib.pyplot as plt

In [ ]:
# ハードウェアを検出し、適切な分散戦略を返します
try:
    # TPUを検出します。TPU_NAME環境変数が設定されている場合、パラメータは必要ありません。
    # これはKaggleで常に当てはまります。
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('TPUで実行中: ', tpu.master())
except ValueError:
    # TPUが検出できなかった場合は、tpuをNoneに設定します
    tpu = None

# TPUが存在する場合
if tpu:
    # TPUクラスタに接続します
    tf.config.experimental_connect_to_cluster(tpu)
    # TPUシステムを初期化します
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # TPUStrategyを使用して分散戦略を設定します
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # デフォルトの分散戦略。CPUや単一のGPUで動作します。
    strategy = tf.distribute.MirroredStrategy()

# 同期中のレプリカ数を出力します
print("REPLICAS: ", strategy.num_replicas_in_sync)  # 現在の分散処理でのレプリカの数を表示します

# トレーニングとテストデータセットのインポート/ロード

In [ ]:
# データセットのパスを設定します
train_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'  # トレーニングデータのパス
test_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'    # テストデータのパス

# データセットをロードします
# 'csv'形式のデータを読み込み、指定したファイルからトレーニングデータを取得します
train_dataset = load_dataset('csv', data_files={'train': train_path})['train']
# テストデータも同様に読み込みます
test_dataset = load_dataset('csv', data_files={'test': test_path})['test']

# テストデータセットからIDを保存します
# 推論や後続の処理で使用するために、テストデータのIDを取得します
test_ids = test_dataset['id']

In [ ]:
# テストデータセットに不足している列を追加します
# モデルの情報や勝者の情報を格納するための列を追加します
for col in ['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie']:
    # 各列がテストデータセットに存在しない場合
    if col not in test_dataset.column_names:
        # 空の値で新しい列を追加します
        test_dataset = test_dataset.add_column(col, [""] * len(test_dataset))

# 列の値をint64型に変換します
for col in ['winner_model_a', 'winner_model_b', 'winner_tie']:
    # トレーニングデータセットに対してマップを適用し、Noneの場合は0に置き換えます
    train_dataset = train_dataset.map(lambda x: {col: int(x[col]) if x[col] is not None else 0})
    # テストデータセットに対しても同様にマップを適用し、空文字列の場合は0に置き換えます
    test_dataset = test_dataset.map(lambda x: {col: int(x[col]) if x[col] != "" else 0})

In [ ]:
# ローカルでbert-base-casedのファイルを使用します
source_dir = '/kaggle/input/huggingface-bert/bert-base-cased'  # BERTモデルのソースディレクトリ

model_dir = '/kaggle/working/bert-base-cased'  # モデルを保存するディレクトリ
# ディレクトリが存在しない場合は作成します
os.makedirs(model_dir, exist_ok=True)

# 必要なファイルをソースディレクトリからモデルディレクトリにコピーします
shutil.copy(os.path.join(source_dir, 'config.json'), model_dir)        # モデルの設定ファイル
shutil.copy(os.path.join(source_dir, 'pytorch_model.bin'), model_dir)  # PyTorch用のモデルファイル
shutil.copy(os.path.join(source_dir, 'tf_model.h5'), model_dir)        # TensorFlow用のモデルファイル
shutil.copy(os.path.join(source_dir, 'tokenizer.json'), model_dir)     # トークナイザの設定ファイル
shutil.copy(os.path.join(source_dir, 'vocab.txt'), model_dir)          # 語彙情報を含むファイル
shutil.copy(os.path.join(source_dir, 'modelcard.json'), model_dir)     # モデルカードのファイル

In [ ]:
# データのトークナイゼーションを行います
# 事前にトレーニングされたBERTトークナイザをモデルディレクトリから読み込みます
tokenizer = BertTokenizer.from_pretrained(model_dir)

# トークナイゼーションを行う関数を定義します
def tokenize_function(examples):
    # 'model_a'と'model_b'の入力に対してトークナイゼーションを行います
    # パディングや切り捨てを行い、最大長を512に設定します
    return tokenizer(examples['model_a'], examples['model_b'], padding="max_length", truncation=True, max_length=512)

# トレーニングデータセットに対してトークナイゼーションを適用します
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)

# テストデータセットに対しても同様にトークナイゼーションを適用します
test_tokenized_datasets = test_dataset.map(tokenize_function, batched=True)

# データをバッチ化するためのコラレーターを作成します
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # トークナイザを使用してパディングを行います

In [ ]:
# 正しい形状のtf.data.Datasetに変換します
def convert_to_tf_dataset(dataset, label_col=None):
    # ラベル列が指定されている場合、ラベル列以外の不要な列を削除します
    if label_col:
        dataset = dataset.remove_columns([col for col in dataset.column_names if col != label_col and col not in tokenizer.model_input_names])
    else:
        # ラベル列が指定されていない場合、モデル入力に必要な列以外を削除します
        dataset = dataset.remove_columns([col for col in dataset.column_names if col not in tokenizer.model_input_names])

    # tf.data.Datasetに変換します
    return dataset.to_tf_dataset(
        columns=tokenizer.model_input_names,  # モデルに必要な入力列
        label_cols=[label_col] if label_col else None,  # ラベル列
        shuffle=True,  # データをシャッフル
        batch_size=64,  # バッチサイズを64に設定
        collate_fn=data_collator  # データをバッチ化するためのコラレーター
    )

# 推論用のtf.data.Datasetを作成します
def convert_to_tf_dataset_for_inference(dataset):
    # モデル入力に必要な列のみを残します
    dataset = dataset.remove_columns([col for col in dataset.column_names if col not in tokenizer.model_input_names])
    # 推論用のデータセットをtf.data.Datasetに変換します
    return dataset.to_tf_dataset(
        columns=tokenizer.model_input_names,  # モデルに必要な入力列
        shuffle=False,  # シャッフルは行わない
        batch_size=16,  # バッチサイズを16に設定
        collate_fn=data_collator  # データをバッチ化するためのコラレーター
    )

In [ ]:
# トークナイゼーションされたデータセットから、トレーニングデータセットを作成します
train_dataset = convert_to_tf_dataset(tokenized_datasets, 'winner_model_a')  # 'winner_model_a'をラベル列として指定
test_labels = 'winner_model_a'  # テストデータのラベルを指定
# テストデータセットを推論用に変換します
test_dataset = convert_to_tf_dataset_for_inference(test_tokenized_datasets)

# トレーニング前にデータ型と形状を確認します
for batch in train_dataset.take(1):  # トレーニングデータセットから最初のバッチを取得します
    inputs, labels = batch  # 入力とラベルを取得します
    print("データ型:")
    print(f"input_ids: {inputs['input_ids'].dtype}, attention_mask: {inputs['attention_mask'].dtype}, labels: {labels.dtype}")
    print("データの形状:")
    print(f"input_ids: {inputs['input_ids'].shape}, attention_mask: {inputs['attention_mask'].shape}, labels: {labels.shape}")  # 各データの形状を表示します

In [ ]:
# カスタムモデルを構築します
class BertLayer(Layer):
    def __init__(self, **kwargs):
        # 親クラスの初期化を行います
        super(BertLayer, self).__init__(**kwargs)
        # ローカルディレクトリから事前トレーニング済みのBERTモデルをロードします
        self.bert = TFBertModel.from_pretrained(model_dir, from_pt=True)

    def call(self, inputs):
        # 入力を受け取ります
        input_ids, attention_mask = inputs
        # BERTモデルに入力を渡し、出力を取得します
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        # 最後の隠れ状態出力を返します
        return outputs.last_hidden_state


def create_keras_model():
    # 入力層を定義します
    input_ids = Input(shape=(512,), dtype=tf.int32, name='input_ids')  # input_ids用の入力
    attention_mask = Input(shape=(512,), dtype=tf.int32, name='attention_mask')  # attention_mask用の入力

    # カスタムBertLayerを介してBERT出力を取得します
    bert_output = BertLayer()([input_ids, attention_mask])
    # 最初のトークンの出力をプールします（文の表現として）
    pooled_output = Lambda(lambda x: x[:, 0], output_shape=(768,))(bert_output)
    # 出力層を追加します（クラス数は4、ソフトマックス活性化関数を使用）
    output = Dense(4, activation='softmax')(pooled_output)

    # モデルを定義します
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    return model  # 作成したモデルを返します

In [ ]:
# モデルの学習を行います
with strategy.scope():  # ストラテジーのスコープ内でモデルを作成します
    model = create_keras_model()  # カスタムKerasモデルを作成します
    # モデルをコンパイルします
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4e-5),  # Adamオプティマイザを使用し、学習率を設定
                  loss='sparse_categorical_crossentropy',  # 損失関数としてスパースカテゴリカルクロスエントロピーを使用
                  metrics=['accuracy'])  # 精度を評価指標として使用

    # モデルのトレーニングを開始します
    history = model.fit(train_dataset, epochs=10, callbacks=[TqdmCallback(verbose=2)])  # 10エポックで学習し、進捗を表示します

In [ ]:
# 予測を行います
predictions = model.predict(test_dataset)  # テストデータセットに対する予測を取得します

# 提出用のDataFrameを作成します
submission = pd.DataFrame({
    'id': test_ids,  # テストIDを含めます
    'winner_model_a': predictions[:, 0],  # モデルAの勝者確率
    'winner_model_b': predictions[:, 1],  # モデルBの勝者確率
    'winner_model_tie': predictions[:, 2]  # 引き分けの確率
})

# データフレームをCSVファイルとして保存します
submission.to_csv('submission.csv', index=False)  # インデックスなしでCSVファイルを作成します

# コメント

> ## ATHviii
> 
> いいですね。私たち二人ともClaudeから助けを受けているようです。
> 
> 